# Predict Football Match Results

In [1]:
import sqlite3
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer

In [2]:
#Database connection
database = sqlite3.connect("database.sqlite")

In [3]:
#I chose English Premiere League
matches = pd.read_sql_query("SELECT season, date, home_team_api_id,away_team_api_id,home_team_goal,away_team_goal FROM Match WHERE league_id is 1729", database)

In [4]:
matches.head()

,season,date,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal
0,2008/2009,2008-08-17 00:00:00,10260,10261,1,1
1,2008/2009,2008-08-16 00:00:00,9825,8659,1,0
2,2008/2009,2008-08-16 00:00:00,8472,8650,0,1
3,2008/2009,2008-08-16 00:00:00,8654,8528,2,1
4,2008/2009,2008-08-17 00:00:00,10252,8456,4,2


In [5]:
matches.shape

(3040, 6)

## Taking Care of Missing Data

In [6]:
#There is no missing data
matches.isnull().sum()

season              0
date                0
home_team_api_id    0
away_team_api_id    0
home_team_goal      0
away_team_goal      0
dtype: int64

## Data Preprocessing

In [7]:
def label_match(row):
    if row["home_team_goal"] > row["away_team_goal"]:
        return "W"
    elif row["home_team_goal"] < row["away_team_goal"]:
        return "L"
    else:
        return "D"

In [8]:
matches['final_result'] = matches.apply (lambda row: label_match(row), axis=1)

In [9]:
#Final result column added based on the score of the match.
matches

,season,date,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,final_result
0,2008/2009,2008-08-17 00:00:00,10260,10261,1,1,D
1,2008/2009,2008-08-16 00:00:00,9825,8659,1,0,W
2,2008/2009,2008-08-16 00:00:00,8472,8650,0,1,L
3,2008/2009,2008-08-16 00:00:00,8654,8528,2,1,W
4,2008/2009,2008-08-17 00:00:00,10252,8456,4,2,W
...,...,...,...,...,...,...,...
3035,2015/2016,2015-10-17 00:00:00,8466,8197,2,2,D
3036,2015/2016,2015-10-19 00:00:00,10003,10194,0,1,L
3037,2015/2016,2015-10-17 00:00:00,8586,8650,0,0,D
3038,2015/2016,2015-10-17 00:00:00,9817,9825,0,3,L


In [10]:
#Some teams have played less match than others.
matches["home_team_api_id"].value_counts()

10260    152
8455     152
9825     152
8586     152
8650     152
10194    152
8456     152
8668     152
10252    152
8472     152
10261    133
8654     133
8659     133
9879     114
8528      95
10003     95
8559      76
8466      76
8655      76
8667      76
9850      76
10172     57
9826      57
8602      57
8658      38
8191      38
8462      38
8197      38
8483      19
9798      19
8549      19
8344      19
8678      19
9817      19
Name: home_team_api_id, dtype: int64

In [11]:
# What is the win rate of home team?
number_of_matches = matches.shape[0]
number_of_home_wins = (matches.final_result == "W").sum()

win_rate = (float(number_of_home_wins) / number_of_matches) * 100
                          

In [12]:
print("Total number of matches: {}".format(number_of_matches))
print("Number of matches won by home team: {}".format(number_of_home_wins))
print("Win rate: {}%".format(win_rate))

Total number of matches: 3040
Number of matches won by home team: 1390
Win rate: 45.723684210526315%


In [13]:
# Type of fields should be integer or float. Therefore I changed the type of result from object to int
matches.dtypes

season              object
date                object
home_team_api_id     int64
away_team_api_id     int64
home_team_goal       int64
away_team_goal       int64
final_result        object
dtype: object

In [14]:
del matches["season"]

In [15]:
matches

,date,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,final_result
0,2008-08-17 00:00:00,10260,10261,1,1,D
1,2008-08-16 00:00:00,9825,8659,1,0,W
2,2008-08-16 00:00:00,8472,8650,0,1,L
3,2008-08-16 00:00:00,8654,8528,2,1,W
4,2008-08-17 00:00:00,10252,8456,4,2,W
...,...,...,...,...,...,...
3035,2015-10-17 00:00:00,8466,8197,2,2,D
3036,2015-10-19 00:00:00,10003,10194,0,1,L
3037,2015-10-17 00:00:00,8586,8650,0,0,D
3038,2015-10-17 00:00:00,9817,9825,0,3,L


In [16]:
matches["target"] = matches["final_result"].astype("category").cat.codes

In [17]:
matches

,date,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,final_result,target
0,2008-08-17 00:00:00,10260,10261,1,1,D,0
1,2008-08-16 00:00:00,9825,8659,1,0,W,2
2,2008-08-16 00:00:00,8472,8650,0,1,L,1
3,2008-08-16 00:00:00,8654,8528,2,1,W,2
4,2008-08-17 00:00:00,10252,8456,4,2,W,2
...,...,...,...,...,...,...,...
3035,2015-10-17 00:00:00,8466,8197,2,2,D,0
3036,2015-10-19 00:00:00,10003,10194,0,1,L,1
3037,2015-10-17 00:00:00,8586,8650,0,0,D,0
3038,2015-10-17 00:00:00,9817,9825,0,3,L,1


In [18]:
# I choose Mancher City as a team for my model
home_matches = matches[matches["home_team_api_id"] == 8456]
away_matches = matches[matches["away_team_api_id"] == 8456]
manchester_city_matches = pd.concat([home_matches, away_matches])
manchester_city_matches

,date,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,final_result,target
35,2008-11-09 00:00:00,8456,8586,1,2,L,1
55,2008-11-22 00:00:00,8456,9825,3,0,W,2
65,2008-11-30 00:00:00,8456,10260,0,1,L,1
85,2008-12-13 00:00:00,8456,8668,0,1,L,1
105,2008-12-26 00:00:00,8456,8667,5,1,W,2
...,...,...,...,...,...,...,...
2932,2016-04-16 00:00:00,8455,8456,0,3,L,1
2955,2016-05-01 00:00:00,8466,8456,4,2,W,2
2977,2016-05-15 00:00:00,10003,8456,1,1,D,0
2991,2015-09-12 00:00:00,9826,8456,0,1,L,1


In [19]:
n_matches_manchester_city = manchester_city_matches.shape[0]
n_wins = (manchester_city_matches.final_result == "W").sum()
win_rate_manchester_city = (float(n_wins) / n_matches_manchester_city) * 100

In [20]:
print("Total number of matches that Manchester City played: {}".format(n_matches_manchester_city))
print("Number of matches won by Manchester City: {}".format(n_wins))
print("Win rate: {}%".format(win_rate))

Total number of matches that Manchester City played: 304
Number of matches won by Manchester City: 160
Win rate: 45.723684210526315%


In [21]:
train, test = train_test_split(manchester_city_matches, test_size=0.2, random_state=25)

In [22]:
print(f"Number of training examples: {train.shape[0]}")
print(f"Number of testing examples: {test.shape[0]}")

Number of training examples: 243
Number of testing examples: 61


In [23]:
rf = RandomForestClassifier(n_estimators=100, min_samples_split=10, random_state=1)

In [24]:
# I trained the model based on only home and away teams' past matches
predictors = ["home_team_api_id", "away_team_api_id"]

In [25]:
rf.fit(train[predictors], train["target"])

RandomForestClassifier(min_samples_split=10, random_state=1)

In [26]:
preds = rf.predict(test[predictors])

In [27]:
acc = accuracy_score(test["target"], preds)

In [28]:
print("Accuracy Score: {}%".format(acc*100))

Accuracy Score: 52.459016393442624%


In [29]:
combined = pd.DataFrame(dict(actual=test["target"], predicted=preds))

## Confusion Matrix

In [30]:
pd.crosstab(index=combined["actual"], columns=combined["predicted"])

predicted,0,1,2
actual,,,
0,2,3,4
1,2,6,9
2,3,8,24


## Precision Score

In [31]:
score = precision_score(test["target"], preds, average="micro")
print("Precision score: {}%".format(score*100))

Precision score: 52.459016393442624%
